In [ ]:
import firedrake
mesh = firedrake.UnitSquareMesh(
    32, 32, diagonal='crossed'
)

In [ ]:
from firedrake import inner, exp, Constant
x = firedrake.SpatialCoordinate(mesh)

ξ_1 = Constant((0.25, 0.25))
ξ_2 = Constant((0.75, 0.75))

r_1 = Constant(1 / 8)
r_2 = Constant(1 / 8)

ϕ_1 = exp(-inner(x - ξ_1, x - ξ_1) / r_1**2)
ϕ_2 = exp(-inner(x - ξ_2, x - ξ_2) / r_2**2)

I = firedrake.Identity(2)
ω = Constant(0.5)
J = firedrake.as_tensor([[0, +ω], [-ω, 0]])

s = Constant(50.0)
U = s * (
    ϕ_1 * (I + J) * (x - ξ_1) -
    ϕ_2 * (I + J) * (x - ξ_2)
)

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, 'CG', 2)
u = firedrake.interpolate(U, V)

In [ ]:
from firedrake import grad, div
Σ = firedrake.TensorFunctionSpace(mesh, 'CG', 2)
F = firedrake.interpolate(grad(U), Σ)

Q = firedrake.FunctionSpace(mesh, 'CG', 2)
div_u = firedrake.interpolate(div(U), Q)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
streamlines = firedrake.streamplot(u, resolution=1/40, axes=axes)
fig.colorbar(streamlines);

In [ ]:
fig, axes = plt.subplots()
colors = firedrake.tripcolor(div_u, cmap='RdBu', axes=axes)
fig.colorbar(colors);

In [ ]:
import numpy as np
import scipy.integrate

xs = np.linspace(0.175, 0.325, 6)
ys = np.linspace(0.3, 0.45, 6)

def f(t, x):
    try:
        return u(x)
    except firedrake.PointNotInDomainError:
        return np.array([0.0, 0.0])
    
def jac(t, x):
    try:
        return F(x)
    except  firedrake.PointNotInDomainError:
        return np.array([[0.0, 0.0], [0.0, 0.0]])

results = []
for x in xs:
    for y in ys:
        result = scipy.integrate.solve_ivp(
            f,
            jac=jac,
            t_span=(0.0, 7.05),
            dense_output=True,
            y0=(x, y),
            method='LSODA',
        )
        results.append(result)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.set_xlim((0, 1))
axes.set_ylim((0, 1))
firedrake.tripcolor(div_u, cmap='RdBu', axes=axes)
for result in results:
    ts = np.linspace(0.0, 7.05, 2001)
    xs = result.sol(ts)
    axes.plot(xs[0, :], xs[1, :], color='k');